<a href="https://colab.research.google.com/github/Nisasoylu/BERT2OME/blob/main/VectorEmbeddingCreation_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importings:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import torch
from transformers import BertTokenizer, BertModel
import numpy as np

Dataset Selection:

In [ ]:
################### HUMAN1 Dataset #######################
test_neg = pd.read_csv(r"test_new_neg_H_N", header = None)
test_pos = pd.read_csv(r"test_new_pos_H_P", header = None)
train_neg = pd.read_csv(r"train_new_neg_H_N", header = None)
train_pos = pd.read_csv(r"train_new_pos_H_P", header = None)

print(test_neg.shape)
print(test_pos.shape)
print(train_neg.shape)
print(train_pos.shape)

x = pd.concat([train_pos, train_neg, test_pos, test_neg], ignore_index=True)

print(x[0][0])
print(x.shape)

train_pos_labels = [1] * len(train_pos)
train_neg_labels = [0] * len(train_neg)
test_pos_labels = [1] * len(test_pos)
test_neg_labels = [0] * len(test_neg)

# Defining labels
y = train_pos_labels + train_neg_labels + test_pos_labels + test_neg_labels
###########################################################

(107, 1)
(40, 1)
(192, 1)
(199, 1)
G C G U G C G G A A U C G A C A U C A A G A G A U U U C G G A A G C A U A A U U U
(538, 1)


In [ ]:
################### HUMAN2 Dataset #######################
rmbase_dataset_ = pd.read_excel("RMBase_800_CD-HIT.xlsx")
y = rmbase_dataset_.label
x = rmbase_dataset_.drop(["label"], axis = 1)
print(len(x))
print(x[0][0])
###########################################################

770
G U C G G A A G C A G G G C C U G G C U C A G C A C C G G G A G G G C G C C G C C


In [ ]:
################### MOUSE Dataset #######################
M = pd.read_excel("M.musculus.xlsx")
y = M.label
x = M.drop(["label"], axis = 1)
print(len(x))
print(x[0][0])
###########################################################

44
A A U G G U U A G C A C U C U G G A C U U U G A A U C C A G C A A U C U G A G U U 


In [ ]:
################### YEAST Dataset #######################
S = pd.read_excel("S.cerevisiae.xlsx")
y = S.label
x = S.drop(["label"], axis = 1)
print(len(x))
print(x[0][0])
###########################################################

278
A A G A C A U C A A U U A G U A C G G G C G U G U G G U C U A G U G G U A U G A U 


## BERT Vector Creation

In [ ]:
x_sentences = []
for i in range(0, len(x)):
    x_sentences.append("[CLS] " + x[0][i] + " [SEP]")

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

x_sentences_tokenized = []
for i in range(0, len(x_sentences)):
    tokenized_text = tokenizer.tokenize(str(x_sentences[i]))
    x_sentences_tokenized.append(tokenized_text)

print(x_sentences_tokenized[0])

x_sentences_indexes = []
for i in range(0, len(x_sentences)):
    x_sentences_indexes.append(tokenizer.convert_tokens_to_ids(x_sentences_tokenized[i]))

x_segment_ids = []
for i in range(0, int(len(x_sentences)/2)):
    x_segment_ids.append([1] * len(x_sentences_tokenized[0]))
    x_segment_ids.append([0] * len(x_sentences_tokenized[0]))

x_tokens_tensor = torch.tensor([x_sentences_indexes])
x_segments_tensors = torch.tensor([x_segment_ids])

model_x = BertModel.from_pretrained("bert-base-uncased", output_hidden_states = True)
model_x.eval()

outputs_x = []
with torch.no_grad():
    outputs_x = model_x(x_tokens_tensor[0], x_segments_tensors[0])
    hidden_states_x = outputs_x[2]

print ("Number of layers:", len(hidden_states_x), "  (initial embeddings + 12 BERT layers)")
layer_i = 0

print ("Number of batches:", len(hidden_states_x[layer_i]))
batch_i = 0

print ("Number of tokens:", len(hidden_states_x[layer_i][batch_i]))
token_i = 0

print ("Number of hidden units:", len(hidden_states_x[layer_i][batch_i][token_i]))

dataset_x = np.array([])
for j in range(0, len(x)):
    a = np.array([])
    for i in range(0,43):
        b = (np.array(hidden_states_x[12][j][i]) + np.array(hidden_states_x[11][j][i]) + np.array(hidden_states_x[10][j][i]) + np.array(hidden_states_x[9][j][i])) / 4
        a = np.hstack((a,b))
    if len(dataset_x) == 0:
        dataset_x = a
    else:
        dataset_x = np.vstack((dataset_x, a))

print(dataset_x.shape)
dataset_x = pd.DataFrame(dataset_x)

print("Data type of x: ",type(x))
print("Data type of y: ", type(y))
print(dataset_x.shape)
print(43*768)
print(len(y))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

['[CLS]', 'a', 'a', 'g', 'a', 'c', 'a', 'u', 'c', 'a', 'a', 'u', 'u', 'a', 'g', 'u', 'a', 'c', 'g', 'g', 'g', 'c', 'g', 'u', 'g', 'u', 'g', 'g', 'u', 'c', 'u', 'a', 'g', 'u', 'g', 'g', 'u', 'a', 'u', 'g', 'a', 'u', '[SEP]']


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Number of layers: 13   (initial embeddings + 12 BERT layers)
Number of batches: 278
Number of tokens: 43
Number of hidden units: 768
(278, 33024)
Data type of x:  <class 'pandas.core.frame.DataFrame'>
Data type of y:  <class 'pandas.core.series.Series'>
(278, 33024)
33024
278


In [ ]:
dataset_x

,0,1,2,3,4,5,6,7,8,9,...,33014,33015,33016,33017,33018,33019,33020,33021,33022,33023
0,-1.033997,-0.257670,-0.145823,-0.078054,-0.653430,0.107204,0.548727,0.411627,-0.464045,-0.636907,...,0.093169,0.037829,-0.219644,-0.174578,0.049885,-0.063594,-0.025446,0.102880,-0.190186,-0.081132
1,0.247758,0.802076,-0.142761,-0.619641,-1.249331,-0.479842,1.144593,-0.297706,-0.739941,-0.639348,...,0.227862,-0.099710,0.086796,0.191353,0.284498,-0.364133,-0.383823,-0.078796,-0.241174,0.526452
2,-1.029269,-0.090777,-0.169259,-0.086926,-0.628812,0.065134,0.471009,0.526354,-0.578674,-0.617457,...,0.086870,0.037960,-0.223002,-0.186475,0.063404,-0.058358,-0.030588,0.107776,-0.187177,-0.073021
3,0.325532,0.995823,-0.323567,-0.672823,-0.969116,-0.609320,1.030457,-0.014639,-0.815730,-0.982193,...,0.381296,-0.090341,-0.083777,0.188543,0.346082,-0.555753,-0.554247,-0.097195,-0.317681,0.401927
4,-0.945901,0.039505,-0.216200,-0.267995,-0.800590,0.086632,0.388623,0.442251,-0.410454,-0.623170,...,0.074589,0.058673,-0.197006,-0.186432,0.077009,-0.058731,-0.038983,0.110766,-0.170416,-0.063440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,0.402175,1.136761,-0.214882,-0.756242,-1.100611,-0.391412,1.260920,-0.670297,0.064047,0.171469,...,-0.571823,-0.330075,-0.006458,0.094664,0.174061,0.285468,-0.531009,0.375750,-0.033589,1.587042
274,-0.875399,-0.060034,-0.136339,-0.313773,-0.739444,-0.090147,0.337031,0.307291,-0.254942,-0.620736,...,0.113140,0.012637,-0.221669,-0.189300,0.063084,-0.076567,-0.031182,0.080489,-0.193069,-0.045033
275,0.192528,0.687492,-0.723959,-0.413793,-0.732941,-0.505630,0.557065,-0.020789,-0.944102,-0.895257,...,0.313657,-0.048479,0.063287,0.079616,0.321181,-0.084663,-0.226378,-0.080681,-0.035426,0.208140
276,-1.018883,0.024974,-0.041305,-0.159999,-0.749059,0.005236,0.521642,0.459444,-0.506034,-0.715803,...,0.089699,0.039876,-0.223312,-0.182021,0.059408,-0.059309,-0.031513,0.106313,-0.184488,-0.079084


In [ ]:
len(y)

278

Save the files:

In [ ]:
################### HUMAN1 Dataset #######################
directory_x = r"BERTHUMAN1EMBEDDINGSX.npy"
np.save(directory_x, dataset_x)

directory_y = r"BERTHUMAN1EMBEDDINGSY.npy"
np.save(directory_y, y)
##########################################################
# Read the files:
a = np.load("BERTHUMAN1EMBEDDINGSX.npy")
b = np.load("BERTHUMAN1EMBEDDINGSY.npy")

In [ ]:
################### HUMAN2 Dataset #######################
directory_x = r"BERTHUMAN2EMBEDDINGSX.npy"
np.save(directory_x, dataset_x)

directory_y = r"BERTHUMAN2EMBEDDINGSY.npy"
np.save(directory_y, y)
##########################################################
# Read the files:
#a = np.load("BERTHUMAN2EMBEDDINGSX.npy")
#b = np.load("BERTHUMAN2EMBEDDINGSY.npy")

In [ ]:
################### MOUSE Dataset #######################
directory_x = r"BERTMOUSEEMBEDDINGSX.npy"
np.save(directory_x, dataset_x)

directory_y = r"BERTMOUSEEMBEDDINGSY.npy"
np.save(directory_y, y)
##########################################################
# Read the files:
#a = np.load("BERTMOUSEEMBEDDINGSX.npy")
#b = np.load("BERTMOUSEEMBEDDINGSY.npy")

In [ ]:
################### YEAST Dataset #######################
directory_x = r"BERTYEASTEMBEDDINGSX.npy"
np.save(directory_x, dataset_x)

directory_y = r"BERTYEASTEMBEDDINGSY.npy"
np.save(directory_y, y)
##########################################################
# Read the files:
#a = np.load("BERTYEASTEMBEDDINGSX.npy")
#b = np.load("BERTYEASTEMBEDDINGSY.npy")